![AuroraAI](images/auroraai-small.png)


# Cluster generation script 

This script performs clustering and factor analysis for a prepared dataset. It assumes three input files, with two of them being mandatory and one optional:

* `METAFILE`: Metadata file in Excel format.
* `VARSFILE`: Preprocessed actual data variables in CSV format
* `BGRVFILE`: Preprocessed background variables in CSV format.

In [ ]:
%matplotlib inline

import os

import numpy as np
from sklearn import __version__
from sklearn import decomposition, manifold
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler

try:
    import umap
except ImportError:
    print('UMAP not available or fails to import, disabling it.')
    umap = None

import pandas as pd
pd.set_option('display.max_columns', None)

from datetime import datetime
from collections import defaultdict

from joblib import dump, load
import json

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from yamlconfig import read_config


## Read and define configuration settings

Select the used dataset:

In [ ]:
DATASET = 'mc'

Read local settings from a YML file:

In [ ]:
config = read_config(verbose=True)
assert DATASET in config, "Selected dataset {} not found in config"
c = config[DATASET]
print('Settings:')
print(c)

Default values for settings:

In [ ]:
LOAD_KMEANS_MODEL, LOAD_FA_MODEL, LOAD_PCA_MODEL = None, None, None
MODELDIR = None
CSV_SEP = ','
INDEX_COL = False # used to be None
POINTSIZE = 5
TRANSPOSE = False
OUTDIR = "."

Apply config values for settings:

In [ ]:

DATADIR = c['datadir']
VARSFILE = c['varsfile']
BGRVFILE = c['bgrvfile']
METAFILE = c['metafile']
if 'index_col' in c:
    INDEX_COL = c['index_col']
if 'n_clusters_kmeans' in c:
    N_CLUSTERS_KMEANS = c['n_clusters_kmeans']
if 'pointsize' in c:
    POINTSIZE = c['pointsize']
if 'transpose' in c:
     TRANSPOSE = c['transpose']
if 'outdir' in c:
    OUTDIR = c['outdir']

In [ ]:
varsfilename = "{}/{}".format(DATADIR, VARSFILE)
metafilename = "{}/{}".format(DATADIR, METAFILE)
bgrvfilename = "{}/{}".format(DATADIR, BGRVFILE)

assert os.path.isfile(varsfilename), "File missing"
assert os.path.isfile(metafilename), "File missing"
assert os.path.isfile(bgrvfilename), "File missing"

if not os.path.isdir(OUTDIR):
    os.mkdir(OUTDIR)
    print('Created directory', OUTDIR)

In [ ]:
def todaystr():
    now = datetime.now()
    return now.strftime("%Y-%m-%d")

## Load data

### Metadata for variables

In [ ]:
df_vars = pd.read_excel(metafilename, index_col=0)
df_vars

In [ ]:
if TRANSPOSE:
    df_vars = df_vars.transpose()
    display(df_vars)

In [ ]:
dimrows = []
dimlist = []
dims = {}
varrow, shortdescrow, descriptionrow, backgroundrow, multiplierrow = None, None, None, None, None
for i, idx in enumerate(df_vars.index):
    if idx.lower().startswith("muuttuja"):
        varrow = i
    if "lyhyt kuvaus" in idx.lower():
        shortdescrow = i
    elif "kuvaus" in idx.lower():
        descriptionrow = i
    if "taustamuuttuja" in idx.lower():
        backgroundrow = i
    if "kerroin" in idx.lower():
        multiplierrow = i
    if idx.startswith("DIM:"):
        dimrows.append(i)
        parts = idx.split(":")
        dimlist.append(parts[1])
        dims[parts[1]] = {'order': len(dims), 'description': parts[2], 'columns': []}
print(varrow, shortdescrow, descriptionrow, backgroundrow, multiplierrow)
if shortdescrow is None:
    shortdescrow = varrow
if descriptionrow is None:
    descriptionrow = varrow
print(dims)
print(dimlist)
print(dimrows)

In [ ]:
varsdict = defaultdict()
varslist = df_vars.columns
for i_c, col in enumerate(varslist):
    #print(col)
    #print(df_vars[col].iloc[descriptionrow])
    varsdict[col] = {'short_description': df_vars[col].iloc[shortdescrow],
                     'description': df_vars[col].iloc[descriptionrow],
                     'background': df_vars[col].iloc[backgroundrow]>0,
                     'multiplier': df_vars[col].iloc[multiplierrow]}
    dimindices = np.argwhere(df_vars[col].iloc[dimrows].fillna(0).values > 0)
    dlist = []
    for d in dimindices:
        dlist.append(dimlist[d[0]])
    varsdict[col]['dimensions'] = dlist
    
with open('{}/variables-{}.json'.format(OUTDIR, todaystr()), 'w', encoding='utf-8') as f:
    json.dump(varsdict, f, ensure_ascii=False, indent=4)

In [ ]:
varsdict

### Actual data variables

In [ ]:
df = pd.read_csv(varsfilename, index_col=INDEX_COL)
df.columns

In [ ]:
for i_c, col in enumerate(df.columns):
    assert col in varslist, col+" not found in variable descriptions"    
    for d in varsdict[col]['dimensions']:
        dims[d]['columns'].append(i_c)

with open('{}/dimensions-{}.json'.format(OUTDIR, todaystr()), 'w', encoding='utf-8') as f:
    json.dump(dims, f, ensure_ascii=False, indent=4)

### Background variables

In [ ]:
df_bg = pd.read_csv(bgrvfilename, index_col=INDEX_COL)
df_bg

## Bring data to numpy, run scaler

In [ ]:
assert df.isnull().sum().sum()==0, "NULLs exists!"

In [ ]:
X = df.values

scaler = StandardScaler()
#scaler = MinMaxScaler()
X = scaler.fit_transform(X)

## k-means

### Run k-means clustering

In [ ]:
if LOAD_KMEANS_MODEL is None:
    kmeans = KMeans(n_clusters=N_CLUSTERS_KMEANS)
    kmeans.fit(X)

    _fn = "kmeans-{}.joblib".format(todaystr())
    dump(kmeans, "{}/{}".format(OUTDIR, _fn))
    print('Wrote', _fn)
else:
    _fn = "{}/{}".format(MODELDIR, LOAD_KMEANS_MODEL)
    assert os.path.isfile(_fn), "File missing: "+_fn
    kmeans = load(_fn)
    print('Loaded', LOAD_KMEANS_MODEL)

In [ ]:
#y = kmeans.labels_
y = kmeans.predict(X)

### Run factor analysis

This is already here although actual FA is done later, as 1st factor needed for sorting the clusters

In [ ]:
n_factors = 4
assert n_factors>3

if LOAD_FA_MODEL is None:
    fa = decomposition.FactorAnalysis(n_components=n_factors, random_state=42)
    X_fa = fa.fit_transform(X)

    _fn = "fa-{}.joblib".format(todaystr())
    dump(fa, "{}/{}".format(OUTDIR, _fn))
    print('Wrote', _fn)
else:
    _fn = "{}/{}".format(MODELDIR, LOAD_FA_MODEL)
    assert os.path.isfile(_fn), "File missing"
    fa = load(_fn)
    print('Loaded', LOAD_FA_MODEL)
    X_fa = fa.transform(X)

orig_cc_fa = fa.transform(kmeans.cluster_centers_)

### Sort clusters

In [ ]:
new_y = np.zeros(len(y), dtype=int)
new_kmeans_cc=np.zeros(kmeans.cluster_centers_.shape)
zipped = list(zip(range(8),list(orig_cc_fa[:,0])))
sorted_clusters = sorted(zipped, key=lambda x: x[1])
clusterdict = {}
for i, c in enumerate(sorted_clusters):
    clusterdict[c[0]] = i
for i,v in enumerate(y):
    new_y[i] = clusterdict[v]
for i in range(new_kmeans_cc.shape[0]):
    new_kmeans_cc[i,:]=kmeans.cluster_centers_[sorted_clusters[i][0],:]

In [ ]:
for c in range(N_CLUSTERS_KMEANS):
    #plt.plot(kmeans.cluster_centers_[c,:]-np.mean(X, axis=0));
    plt.plot(new_kmeans_cc[c,:]-np.mean(X, axis=0));

In [ ]:
df_kmeans_centers = pd.DataFrame(new_kmeans_cc, columns=df.columns)
df_kmeans_centers.index = np.arange(1, len(df_kmeans_centers)+1)
df_kmeans_centers.to_pickle("{}/kmeans_centers-{}.pkl".format(OUTDIR, todaystr()))
df_kmeans_centers.to_csv("{}/kmeans_centers-{}.csv".format(OUTDIR, todaystr()), sep=";")
df_kmeans_centers

### Print most significant variables for each cluster

In [ ]:
_uniq, _counts = np.unique(new_y, return_counts=True)
yc = dict(zip(_uniq, _counts))

In [ ]:
for c in range(N_CLUSTERS_KMEANS):
    #diff = kmeans.cluster_centers_[c,:] #-np.mean(X, axis=0)
    diff = new_kmeans_cc[c,:] #-np.mean(X, axis=0)
    absdiff = np.abs(diff)
    #sorted = np.flip(np.sort(absdiff))
    sorted_indices = np.flip(np.argsort(absdiff))
    print('\n### K-MEANS {} ({:.1f}%) ###'.format(c+1, 100*yc[c]/len(new_y)))
    for i in range(10):
        var = df.columns[sorted_indices[i]]
        print("{:.2f} {} ({})".format(diff[sorted_indices[i]],
                                      var, varsdict[var]['description']))

In [ ]:
kmeans_labels = ['Klusteri '+str(c) for c in range(1,N_CLUSTERS_KMEANS+1)]
kmeans_descriptions = ['Klusteri '+str(c) for c in range(1,N_CLUSTERS_KMEANS+1)]

### Clusterwise means for both clustering and background variables

In [ ]:
dfs, dfs_bg = [], []
for c in range(N_CLUSTERS_KMEANS):
    dfs.append(df.iloc[new_y==c].mean())
    dfs_bg.append(df_bg.iloc[new_y==c].mean(numeric_only=True))
df_means = pd.concat(dfs, axis=1).transpose()
df_bg_means = pd.concat(dfs_bg, axis=1).transpose()

df_means.to_pickle("{}/kmeans_muuttujat-{}.pkl".format(OUTDIR, todaystr()))
df_means.to_csv("{}/kmeans_muuttujat-{}.csv".format(OUTDIR, todaystr()), sep=";")

df_bg_means.to_pickle("{}/kmeans_taustamuuttujat-{}.pkl".format(OUTDIR, todaystr()))
df_bg_means.to_csv("{}/kmeans_taustamuuttujat-{}.csv".format(OUTDIR, todaystr()), sep=";")

### Plot a Stiglitz dimension for all the clusters

In [ ]:
stigvals, stigstrip = [], []

for d in dims.keys():
    ndim = dims[d]['order']
    stigvals.append([])
    stigstrip.append([])
    for c in range(N_CLUSTERS_KMEANS):
        X_cluster = X[new_y==c]
        n_vals, stigval = 0, 0.0
        ssx, ssy, ssv = [], [], []
        for dc in dims[d]['columns']:
            dv = df.columns[dc]
            val = X_cluster[:,dc].mean()*varsdict[dv]['multiplier']
            ssx.append(ndim)
            ssy.append(val)
            ssv.append(dv)
            #print(ssx, ssy)
            stigval += val
            n_vals += 1
            #print("{}: {:.2f}".format(sv, val))
        
        mean_stigval = 0.0
        if n_vals>0:
            mean_stigval = stigval/n_vals
        stigvals[ndim].append(mean_stigval)
        stigstrip[ndim].append((ssx,ssy,ssv))
        #print("AVERAGE: {} {:.2f}".format(c+1, stigval/n_vals))

We can inspect variables for a certain Stiglitz and cluster:

In [ ]:
#stigstrip[6][4]

In [ ]:
for d in dims.keys():
    ndim = dims[d]['order']
    plt.figure(figsize=(8,8))
    ax = plt.gca()
    colors=3*(np.array(stigvals[ndim])<0).astype(int)
    sx, sy = [], []
    for i, ss in enumerate(stigstrip[ndim]):
        sx = sx + [i+1]*len(ss[1])
        sy = sy + ss[1]
    plt.bar(list(range(len(stigvals[ndim]))), stigvals[ndim], 
            color=[sns.color_palette()[c] for c in colors])
    if len(sx):
        sns.stripplot(x=sx, y=sy, size=8, color=sns.color_palette()[2], linewidth=1)
    plt.plot([-1, N_CLUSTERS_KMEANS], [0,0])
    plt.xlim([-0.5, N_CLUSTERS_KMEANS-0.5])
    plt.ylim([-1.95, 1.95])
    plt.title('"{}" eri klustereissa'.format(dims[d]['description']),
              fontsize='large');
    plt.savefig("{}/stiglitz2-{}.png".format(OUTDIR, d), bbox_inches='tight')
    plt.show()

### Process all Stiglitz dimensions for a cluster

#### Plot figures

In [ ]:
stigvecs = []
for c in range(N_CLUSTERS_KMEANS):
    stigvec = []
    for ndim in range(len(dims)):
        stigvec.append(stigvals[ndim][c])
    stigvecs.append(stigvec)    
    sx, sy = [], []
    for i, ss in enumerate(stigstrip):
        sx = sx + ss[c][0]
        sy = sy + ss[c][1]
        #print(ss[0][0])
        #if c==0:
            #print(ss[0][2])
    if 1:
        plt.figure(figsize=(16,5))
        ax = plt.gca()
        colors=3*(np.array(stigvec)<0).astype(int)
        plt.bar(range(len(stigvec)), np.tanh(stigvec),
            color=[sns.color_palette()[co] for co in colors])
        sns.stripplot(x=sx, y=np.tanh(sy), size=8, color=sns.color_palette()[2], linewidth=1)
        plt.plot([-1, len(dims)], [0,0])
        plt.xticks(list(range(len(dims))), dims.keys(),
                fontsize='small')
        plt.xlim([-0.5, len(dims)-0.5])
        plt.ylim([-1.05, 1.05])
        plt.title('Stiglitz-ulottuvuudet klusterissa {}'.format(c+1), fontsize='x-large');
        plt.savefig("{}/stiglitz-klusteri-{}.png".format(OUTDIR, c+1), bbox_inches='tight')
        plt.show()

#### Save CSVs

In [ ]:
df_dcl = pd.DataFrame(np.array(stigvecs), columns=dims.keys())
df_dcl["klusteri"] = range(1,N_CLUSTERS_KMEANS+1)
df_dcl = df_dcl.set_index("klusteri")
prop = []
for c in range(N_CLUSTERS_KMEANS):
    prop.append(yc[c]/len(new_y))
df_dcl["osuus"] = prop
df_dcl["koko"] = [yc[c] for c in range(N_CLUSTERS_KMEANS)]

df_dcl.to_csv("{}/klusterit-{}.csv".format(OUTDIR, todaystr()))

df_dcl

In [ ]:
with open("{}/klusterimuuttujat2-{}.csv".format(OUTDIR, todaystr()), 'w') as f:
    f.write("klusteri;x;y;x_noise;variable\n")
    for s in range(len(stigstrip)):
        for i_c, c in enumerate(stigstrip[s]):
            for l in range(len(c[0])):
                f.write("{};{};{};{};{}\n".format(i_c+1, c[0][l], c[1][l],
                                                  c[0][l]+np.random.normal(loc=0.0, scale=0.03),
                                                  c[2][l]))

## Common plot functions

These are common to FA, PCA, UMAP

In [ ]:
#DEFAULT_COLORS = ["#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E"]
DEFAULT_COLORS = sns.color_palette("bright")

import matplotlib.patheffects as PathEffects

def plot_embedding(X, CC=None, title=None, cov_ell=None,
                   coloring=None, n_CC=None, palette=None, 
                   remove_nans=False, sizes=None, counts=None,
                   save_as=None, xylabels=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)
    if CC is not None:
        CC = (CC - x_min) / (x_max - x_min)
    if n_CC is None:
        n_CC = N_CLUSTERS_KMEANS
    if coloring is None:
        coloring = y
    if remove_nans:
        notnan = ~np.isnan(coloring)
        X = X[notnan,:]
        coloring = coloring[notnan]
        if sizes is not None and not isinstance(sizes, (int, float)):
            sizes = sizes[notnan]
    if sizes is None:
        sizes = 1.0
    if palette is None:
        colors = DEFAULT_COLORS
    else:
        colors = palette
    
    plt.figure(figsize=(18,10))
    ax = plt.gca()
    #plt.axis('off')
    if xylabels is not None:
        plt.xlabel(xylabels[0], fontsize='x-large', fontweight='bold')
        plt.ylabel(xylabels[1], fontsize='x-large', fontweight='bold')
    #ax.xaxis.set_visible(False)
    plt.setp(ax.spines.values(), visible=False)
    ax.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)
    ax.patch.set_visible(False)
    
    #plt.scatter(X[:,0], X[:,1], s=sizes,
    #            color=[plt.cm.Set1(int(yi) / 10.) for yi in coloring])
    plt.scatter(X[:,0], X[:,1], s=sizes,
                color=[colors[int(yi)] for yi in coloring])
    if cov_ell is not None:
        for i in range(n_CC):
            XX = X[coloring==i]
            confidence_ellipse(XX[:,0], XX[:,1], ax, n_std=cov_ell,
                               edgecolor='k', lw=4)
            confidence_ellipse(XX[:,0], XX[:,1], ax, n_std=cov_ell,
                               edgecolor=colors[int(i)], lw=2)
    if CC is not None:
        clusters = range(n_CC)
        plt.scatter(CC[:,0], CC[:,1], s=200.0, 
                    c=[colors[int(yi)] for yi in clusters],
                    edgecolors='k', lw=2)
        for i in clusters:
            if i==-1: # if some cluster number needs to be below the centroid
                xytext = (-6, -35)
            else:
                xytext = (-6, 20)
            plt.annotate("{}".format(i+1),
                         (CC[i,0], CC[i,1]),
                         xytext=xytext, textcoords='offset points',
                         fontsize=28, #'xx-large', 
                         fontweight='bold',
                         bbox=dict(facecolor='white', edgecolor='black', lw=2, alpha=0.7,
                                   boxstyle='round,pad=0.08'))
    if counts is not None:
        for i in counts.keys():
            if ~np.isnan(i):
                i = int(i)
                plt.annotate("{}: {}".format(i+1, counts[i]), 
                             (0.8, 0.3-0.05*i), c = colors[i], #plt.cm.Set1((i+1)/10.),
                             fontsize='xx-large', fontweight='bold')  
    if title is not None:
        plt.title(title, fontsize='xx-large', fontweight='bold')
    if save_as is not None:
        plt.savefig(save_as, bbox_inches='tight')

In [ ]:
def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = mpl.patches.Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = mpl.transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)


## Factor analysis

In [ ]:
cc_fa = fa.transform(new_kmeans_cc)

In [ ]:
for c in range(n_factors):
    factor = fa.components_[c,:]
    absfactor = np.abs(factor)
    sorted_indices = np.flip(np.argsort(absfactor))
    print('\n### FA {} ###'.format(c+1))
    for i in range(10):
        var = df.columns[sorted_indices[i]]
        print("{:.2f} {} ({})".format(factor[sorted_indices[i]],
                                      var, varsdict[var]['description']))

In [ ]:
with open("{}/faktorit-{}.csv".format(OUTDIR, todaystr()), 'w') as f:
    f.write("faktori1,faktori2,faktori3,faktori4,klusteri\n")
    for i in range(len(X_fa)):
        f.write("{},{},{},{},{}\n".format(X_fa[i,0],X_fa[i,1],
                                          X_fa[i,2],X_fa[i,3],
                                          new_y[i]+1))

In [ ]:
fa_labels = ['Faktori 1', 'Faktori 2', 'Faktori 3', 'Faktori 4']

In [ ]:
data = {"clusters": kmeans_labels,
        "cluster_descriptions": kmeans_descriptions,
        "factors": fa_labels}
with open('{}/descriptions-{}.json'.format(OUTDIR, todaystr()), 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
plot_embedding(X_fa, cc_fa, title="FA klusterit", cov_ell=1.0, coloring=new_y,
               save_as="{}/fa-12.png".format(OUTDIR), sizes=POINTSIZE, counts=yc, xylabels=fa_labels)

In [ ]:
plot_embedding(X_fa[:,[2,3]], cc_fa[:,[2,3]], title="FA klusterit", cov_ell=1.0, coloring=new_y,
               save_as="{}/fa-34.png".format(OUTDIR), sizes=POINTSIZE, counts=yc, xylabels=[fa_labels[i] for i in [2,3]])

## PCA


In [ ]:
n_components = 10
assert n_components>3

if LOAD_PCA_MODEL is None:
    pca = decomposition.PCA(n_components=n_components, whiten=False)
    X_pca = pca.fit_transform(X)

    _fn = "pca-{}.joblib".format(todaystr())
    dump(pca, "{}/{}".format(OUTDIR, _fn))
    print('Wrote', _fn)
else:
    _fn = "{}/{}".format(MODELDIR, LOAD_PCA_MODEL)
    assert os.path.isfile(_fn), "File missing"
    pca = load(_fn)
    print('Loaded', LOAD_PCA_MODEL)
    X_pca = pca.transform(X)

plt.figure()
plt.bar(np.arange(n_components)+1, 
        pca.explained_variance_ratio_)
plt.title('Explained variance by PCA components')
plt.ylabel('explained variance')
plt.xlabel('PCA component');

In [ ]:
cc_pca = pca.transform(new_kmeans_cc)

In [ ]:
for c in range(n_components):
    comp = pca.components_[c,:]
    abscomp = np.abs(comp)
    sorted_indices_pca = np.flip(np.argsort(abscomp))
    print('\n### PCA {} ###'.format(c+1))
    for i in range(10):
        var = df.columns[sorted_indices_pca[i]]
        print("{:.2f} {} ({})".format(comp[sorted_indices_pca[i]],
                                      var, varsdict[var]['description']))

In [ ]:
with open("{}/pca-{}.csv".format(OUTDIR, todaystr()), 'w') as f:
    f.write("pca1,pca2,pca3,pca4,klusteri\n")
    for i in range(len(X_pca)):
        f.write("{},{},{},{},{}\n".format(X_pca[i,0],X_pca[i,1],
                                          X_pca[i,2],X_pca[i,3],
                                          new_y[i]+1))

In [ ]:
pca_labels = ['PCA 1', 'PCA 2', 'PCA 3', 'PCA 4']

In [ ]:
data_pca = {"clusters": kmeans_labels,
            "cluster_descriptions": kmeans_descriptions,
            "pca_components": pca_labels}
with open('{}/descriptions-pca-{}.json'.format(OUTDIR, todaystr()), 'w', encoding='utf-8') as f:
    json.dump(data_pca, f, ensure_ascii=False, indent=4)

In [ ]:
plot_embedding(X_pca, cc_pca, title="PCA klusterit", cov_ell=1.0, coloring=new_y,
               save_as="{}/pca-12.png".format(OUTDIR), sizes=POINTSIZE, counts=yc, xylabels=pca_labels)

In [ ]:
plot_embedding(X_pca[:,[2,3]], cc_pca[:,[2,3]], title="PCA klusterit", cov_ell=1.0, coloring=new_y,
               save_as="{}/pca-34.png".format(OUTDIR), sizes=POINTSIZE, counts=yc, xylabels=[pca_labels[i] for i in [2,3]])

## UMAP

In [ ]:
if umap is not None:
    n_neighbors = 20
    min_dist = 0.5 # 0.5 # 0.1
    umapmodel = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist)
    #noise = np.random.normal(0, .5, X.shape)
    X_umap = umapmodel.fit_transform(X)#+noise)
    cc_umap = umapmodel.transform(new_kmeans_cc)

In [ ]:
if umap is not None:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,3))
    ax1.hist(X_umap[:,0],50)
    ax1.set_title('x')

    ax2.hist(X_umap[:,1], 50)
    ax2.set_title('y');

In [ ]:
if umap is not None:
    X_umap_filt = X_umap[X_umap[:,0]<0]
    y_filt = y[X_umap[:,0]<0]
    #imm_filt = imm[X_umap[:,0]<0]
    #immsizes_filt = immsizes[X_umap[:,0]<0]

In [ ]:
if umap is not None:
    plot_embedding(X_umap, cc_umap,
                   "UMAP projection with n_neighbors=%d, min_dist=%.2f" % (n_neighbors,
                                                                           min_dist),
                   save_as="{}/umap.png".format(OUTDIR), sizes=POINTSIZE, coloring=new_y)